In [ ]:
# 초기코드 - python
from keras.utils import np_utils
from keras.layers import Dense
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
#from PIL import Image
import os, glob, numpy as np
from keras.models import load_model
import random
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Activation, Dense
from tensorflow import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# 새 섹션

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/LPD_competition.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/968/32.jpg        
   creating: train/486/
  inflating: train/486/21.jpg        
  inflating: train/486/22.jpg        
  inflating: train/486/23.jpg        
  inflating: train/486/12.jpg        
  inflating: train/486/13.jpg        
  inflating: train/486/14.jpg        
  inflating: train/486/15.jpg        
  inflating: train/486/16.jpg        
  inflating: train/486/17.jpg        
  inflating: train/486/18.jpg        
  inflating: train/486/19.jpg        
  inflating: train/486/20.jpg        
  inflating: train/486/9.jpg         
  inflating: train/486/10.jpg        
  inflating: train/486/11.jpg        
  inflating: train/486/0.jpg         
  inflating: train/486/1.jpg         
  inflating: train/486/2.jpg         
  inflating: train/486/3.jpg         
  inflating: train/486/4.jpg         
  inflating: train/486/5.jpg         
  inflating: train/486/6.jpg         
  inflating: train/486/7.jpg         
  inflating: train/486/8.jpg

In [ ]:
train_data_dir = '/content/train'
img_height=299
img_width=299
batch_size=64

train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20, #바꿔볼것
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=(0.9,1.1),
    validation_split=0.2) # set validation split

val_datagen = ImageDataGenerator(rescale=1./255,
                                 validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    interpolation='bicubic',
    shuffle=True,
    subset='training') # set as training data

validation_generator = val_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    interpolation='bicubic',
    shuffle=False,
    subset='validation') # set as validation data

Found 39000 images belonging to 1000 classes.
Found 9000 images belonging to 1000 classes.


# 새 섹션

In [ ]:
model = InceptionResNetV2(include_top=True)
model.summary()

96116736/96112376 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
______________________

# 새 섹션

In [ ]:
#optimizer = SGD(lr=1e-4, momentum=0.9)
#optimizer RMSprop(lr=2e-5)
#optimizer = keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = './model'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + '/inception_resnetV2_weights.model'
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [ ]:
nb_epochs=50
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    callbacks=[checkpoint,early_stopping]
    )
hist_df = pd.DataFrame(history.history) 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
!cp history.csv /content/gdrive/MyDrive/history.csv
!cp -r model /content/gdrive/MyDrive/model

Epoch 1/50
609/609 [==============================] - 1150s 2s/step - loss: 5.8700 - accuracy: 0.0773 - val_loss: 2.6203 - val_accuracy: 0.3515

Epoch 00001: val_loss improved from inf to 2.62033, saving model to ./model/inceptionv3_weights.model
INFO:tensorflow:Assets written to: ./model/inceptionv3_weights.model/assets
Epoch 2/50
609/609 [==============================] - 1099s 2s/step - loss: 0.8519 - accuracy: 0.7751 - val_loss: 0.5215 - val_accuracy: 0.8467

Epoch 00002: val_loss improved from 2.62033 to 0.52147, saving model to ./model/inceptionv3_weights.model
INFO:tensorflow:Assets written to: ./model/inceptionv3_weights.model/assets
Epoch 3/50
609/609 [==============================] - 1093s 2s/step - loss: 0.3124 - accuracy: 0.9086 - val_loss: 0.5391 - val_accuracy: 0.8382

Epoch 00003: val_loss did not improve from 0.52147
Epoch 4/50
609/609 [==============================] - 1086s 2s/step - loss: 0.2117 - accuracy: 0.9377 - val_loss: 0.2275 - val_accuracy: 0.9277

Epoch 000

# 새 섹션

In [ ]:
#또는
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

!cp history.json ./gdrive/MyDrive/inception_resnetV2_history.json
!cp history.csv ./gdrive/MyDrive/inception_resnetV2_history.csv

In [ ]:
!mkdir temp
!mv ./test/* ./temp
!mkdir ./test/unknown
!mv ./temp/* ./test/unknown/

In [ ]:
test_data_dir = '/content/test'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size = (img_height, img_width),
    batch_size=batch_size,
    class_mode=None, #결과 똑같이나온다 None이나 Categorical 이나
    color_mode='rgb',
    interpolation='bicubic',
    shuffle=False
    )

Found 72000 images belonging to 1 classes.


In [ ]:
co=0
st=train_generator.class_indices.keys()
result_arr=list(st)
print(result_arr)

['0', '1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '21', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '22', '220', '221', '222', '223', '224', '225', '226', '227', '228', '22

In [ ]:
# !rm -rf /content/test/unknown
# !mkdir /content/test/unknown

In [ ]:
model = load_model('/content/gdrive/MyDrive/model/model/inception_resnetV2_weights.model')
prediction = model.predict(test_generator)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
predicted_class = np.argmax(prediction,axis=1)

In [ ]:
test = pd.read_csv('/content/sample.csv')
df = test.reindex(index=test.index[:72000], columns=list(test.columns))
print(df)

In [ ]:
co=0
for i in predicted_class :
    ans = i
    ans=result_arr[ans]
    ans=int(ans)
    fname =test_generator.filenames[co]
    fname = fname.replace("unknown/","")
    find_row=test.loc[(test['filename']==fname)]
    find_row=int(find_row.index.tolist()[0])
    df.loc[find_row,'prediction']=ans
    co+=1
    print(ans)

In [ ]:
df.to_csv("result.csv",mode='w',index=False)
!cp result.csv ./gdrive/MyDrive/inception_resnetV2.csv

# 새 섹션